In [1]:
!git clone https://github.com/exowanderer/vaelstmpredictor

Cloning into 'vaelstmpredictor'...
remote: Enumerating objects: 227, done.
remote: Counting objects: 100% (227/227), done.
remote: Compressing objects: 100% (133/133), done.
remote: Total 3428 (delta 154), reused 148 (delta 93), pack-reused 3201
Receiving objects: 100% (3428/3428), 15.34 MiB | 20.00 MiB/s, done.
Resolving deltas: 100% (2429/2429), done.


In [2]:
!pip install paramiko

     |████████████████████████████████| 204kB 2.8MB/s 
     |████████████████████████████████| 2.3MB 42.0MB/s 
     |████████████████████████████████| 61kB 22.3MB/s 
     |████████████████████████████████| 768kB 37.4MB/s 
     |████████████████████████████████| 102kB 28.6MB/s 


In [0]:
import os
os.chdir('/content/vaelstmpredictor')

In [4]:
!git checkout regression

Branch 'regression' set up to track remote branch 'regression' from 'origin'.
Switched to a new branch 'regression'


In [5]:
!git branch

  master
* regression


In [6]:
!git pull

Already up to date.


In [7]:
pwd

'/content/vaelstmpredictor'

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
ls /content/drive/My\ Drive/

In [16]:
os.mkdir(os.environ['HOME'] + '.vaelstmpredictor/')

FileExistsError: ignored

In [17]:
!rsync -rPvua /content/drive/My\ Drive/data /root/.vaelstmpredictor/

sending incremental file list
created directory /root/.vaelstmpredictor
data/
data/exoplanet_spectral_database.joblib.save
  2,165,842,183 100%   68.13MB/s    0:00:30 (xfr#1, to-chk=1/3)
data/exoplanet_spectral_database_normalized.joblib.save
  2,165,842,226 100%   61.80MB/s    0:00:33 (xfr#2, to-chk=0/3)

sent 4,332,742,170 bytes  received 104 bytes  67,174,298.82 bytes/sec
total size is 4,331,684,409  speedup is 1.00


In [23]:
ls /root/.vaelstmpredictor/data

exoplanet_spectral_database.joblib.save
exoplanet_spectral_database_normalized.joblib.save


In [24]:
from TrainChromosome_Conv1D import *

Using TensorFlow backend.


In [0]:
hostname = 'colab'

In [0]:
class blankClass(object):
	def __init__(self):
		pass

In [0]:
clargs = blankClass()

In [0]:
while True:
    try:
        info_message('Grabbing Untrained Chromosome')
        chromosome = requests.get(
            url="http://{}/GetUnTrainedChrom".format(base_url))
    except:
        chromosome = None

    if(chromosome != None and chromosome.text == "0"):
        info_message("No more Chromosomes to train")
        sleep(30)
    elif(chromosome != None):
        try:
            info_message('Accessing Params from Untrained Chromosome')
            params = chromosome.json()
        except:
            continue

        clargs.batch_size = params["batch_size"]
        clargs.chromosomeID = params["chromosomeID"]
        clargs.cross_prob = params["cross_prob"]
        clargs.dnn_kl_weight = params["dnn_kl_weight"]
        clargs.dnn_log_var_prior = params["dnn_log_var_prior"]
        clargs.dnn_weight = params["dnn_weight"]
        clargs.fitness = params["fitness"]
        clargs.generationID = params["generationID"]
        clargs.isTrained = params["isTrained"]
        clargs.kl_anneal = params["kl_anneal"]
        clargs.log_dir = params["log_dir"]
        clargs.do_log = True
        clargs.do_ckpt = False
        clargs.verbose = True
        clargs.save_model = False
        #clargs.model_dir = params["model_dir"]
        clargs.model_dir = "data/models"
        clargs.mutate_prob = params["mutate_prob"]
        clargs.num_dnn_layers = params["num_dnn_layers"]
        clargs.num_epochs = params["num_epochs"]
        clargs.num_generations = params["num_generations"]
        clargs.num_vae_layers = params["num_vae_layers"]
        clargs.optimizer = params["optimizer"]
        clargs.patience = params["patience"]
        clargs.population_size = params["population_size"]
        clargs.prediction_log_var_prior = params[
            "prediction_log_var_prior"]
        clargs.predictor_type = 'regression'  # params["predictor_type"]
        clargs.run_name = params["run_name"]
        clargs.size_dnn_hidden = params["size_dnn_hidden"]
        clargs.size_vae_hidden = params["size_vae_hidden"]
        clargs.vae_latent_dim = params["size_vae_latent"]
        clargs.table_dir = params["table_dir"]
        clargs.train_file = params["train_file"]  # 'mnist'  #
        clargs.vae_kl_weight = params["vae_kl_weight"]
        clargs.vae_weight = params["vae_weight"]
        clargs.w_kl_anneal = params["w_kl_anneal"]
        clargs.num_conv_layers = params["num_conv_layers"]
        clargs.size_kernel = params["size_kernel"]
        clargs.size_pool = params["size_pool"]
        clargs.size_filter = params["size_filter"]

        clargs.hostname = hostname
        clargs.time_stamp = int(time())

        vae_hidden_dims = [clargs.size_vae_hidden] * clargs.num_vae_layers
        dnn_hidden_dims = [clargs.size_dnn_hidden] * clargs.num_dnn_layers
        size_kernel = np.array(json.loads(clargs.size_kernel))
        size_pool = np.array(json.loads(clargs.size_pool))
        size_filter = np.array(json.loads(clargs.size_filter))

        if clargs.train_file == 'exoplanet':
            from vaelstmpredictor.utils.data_utils import ExoplanetData
            data_instance = ExoplanetData(train_file=None,
                                          batch_size=clargs.batch_size)
        elif clargs.train_file == 'mnist':
            from vaelstmpredictor.utils.data_utils import MNISTData
            data_instance = MNISTData(batch_size=clargs.batch_size)
        else:
            raise Exception(
                "clargs.train_file must be either `exoplanet` or `mnist`")
        _, n_features = data_instance.data_train.shape
        _, n_features = data_instance.data_valid.shape
        print(data_instance.train_labels.shape)
        _, n_labels = data_instance.train_labels.shape
        n_channels = 1

        data_shape = (n_features, n_channels)

        clargs.original_dim = n_features
        clargs.n_labels = n_labels
        # len(np.unique(data_instance.train_labels))

        chrom_params = {}
        chrom_params['data_instance'] = data_instance
        chrom_params['verbose'] = clargs.verbose
        chrom_params['save_model'] = clargs.save_model
        chrom_params['vae_hidden_dims'] = vae_hidden_dims
        chrom_params['dnn_hidden_dims'] = dnn_hidden_dims
        chrom_params['vae_latent_dim'] = clargs.vae_latent_dim
        chrom_params['generationID'] = clargs.generationID
        chrom_params['chromosomeID'] = clargs.chromosomeID
        chrom_params['vae_weight'] = clargs.vae_weight
        chrom_params['vae_kl_weight'] = clargs.vae_kl_weight
        chrom_params['dnn_weight'] = clargs.dnn_weight
        chrom_params['dnn_kl_weight'] = clargs.dnn_kl_weight
        chrom_params['num_conv_layers'] = clargs.num_conv_layers
        chrom_params['size_kernel'] = size_kernel
        chrom_params['size_pool'] = size_pool
        chrom_params['size_filter'] = size_filter
        chrom_params['clargs'] = clargs
        chrom_params['data_shape'] = data_shape

        info_message('\n\nParams for this VAE_NN:')
        for key, val in clargs.__dict__.items():
            print('{:20}{}'.format(key, val))

        info_message("Training Chromosome " + str(clargs.chromosomeID) +
                     " Generation " + str(clargs.generationID))
        try:
            K.clear_session()
            chromosome = Chromosome(**chrom_params)
            chromosome.verbose = True

            start_time = time()

            if clargs.verbose:
                info_message('Start Training: {}'.format(start_time))

            chromosome.model.summary()
            # break
            chromosome.train(verbose=True)
            K.clear_session()

            end_time = time()
            run_time = end_time - start_time

            if clargs.verbose:
                info_message('End Training: {}'.format(end_time))
                info_message('Runtime Training: {}'.format(run_time))

        except Exception as e:
            warning_message("Error has occured while training")
            print(e)
            chromosome = None
            continue

        info_message('\n')
        print('Result: ', end=" ")
        print('GenerationID: {}'.format(chromosome.generationID), end=" ")
        print('ChromosomeID: {}'.format(chromosome.chromosomeID), end=" ")
        print('Fitness: {}\n'.format(chromosome.fitness))

        params["isTrained"] = 2
        params["fitness"] = chromosome.fitness
        params["val_vae_reconstruction_loss"] = chromosome.best_loss[
            'val_vae_reconstruction_loss']
        params["val_vae_latent_args_loss"] = chromosome.best_loss[
            'val_vae_latent_args_loss']
        params["val_dnn_latent_layer_loss"] = chromosome.best_loss[
            'val_dnn_latent_layer_loss']
        params["val_dnn_latent_mod_loss"] = chromosome.best_loss[
            'val_dnn_latent_mod_loss']
        params["hostname"] = clargs.hostname
        params["start_time"] = start_time
        params["end_time"] = end_time
        params["run_time"] = run_time

        sent = False
        while(not sent):
            try:
                resp = requests.get(
                    url="http://{}/AddChrom".format(base_url),
                    params=params)
                info_message("Response: " + resp.text)
                sent = (resp.text == "1")
            except:
                sent = False
